## Import Library

In [123]:
import cv2
import mediapipe as mp
import numpy as np
import PIL
import os
import copy
import itertools
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img


## Data Augmentation

In [139]:
#Parameter
src_path = "dataset/slideleft/"
aug_path = "tmp/augmented_image/slideleft/"
img_dir = os.listdir(src_path)

In [140]:
#Create augmented picture

for file in img_dir:
        datagen = ImageDataGenerator(
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                brightness_range= [0.008,0.1],
                fill_mode='nearest')

        img = load_img(os.path.join(src_path,file))  # this is a PIL image
        x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

        # the .flow() command below generates batches of randomly transformed images
        # and saves the results to the `preview/` directory
        i = 0
        for batch in datagen.flow(x, batch_size=1,
                                save_to_dir=aug_path, save_prefix='aug', save_format='png'):
                i += 1
                if i > 7:
                        break  # otherwise the generator would loop indefinitely

## Define Pre-Requisite

### Define mediapipe hand detector

In [141]:
#mediapipe detector
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

### Function to PreProcess landmark

In [142]:
#get coordinate box of hand
def calc_bounding_rect(image, landmarks): 
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv2.boundingRect(landmark_array)

    return [x, y, x + w, y + h]

#get coordinate of x and y of each keypoints of hands
def calc_landmark_list(image, landmarks): 
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point


#convert coordinate of keypoints into respective of bottom of palm (0,0)
def pre_process_landmark(landmark_list): 
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

### Function for drawing

In [143]:
def draw_bounding_rect(use_brect, image, brect):
    if use_brect:
        # Outer rectangle
        cv2.rectangle(image, (brect[0], brect[1]), (brect[2], brect[3]),
                     (0, 0, 0), 1)

    return image


### Function to append keypoints into csv

In [144]:
def append_to_csv(filename, listvalue):
    if os.path.isfile(filename):
        with open(filename, 'a+', newline="") as write_obj:
            csv_writer= writer(write_obj)
            csv_writer.writerow(listvalue)
    else:
        return "Create your empty csv format file first!!"

## Detect hand keypoints from picture

In [183]:
#Parameter
data_path = "junk/temp2/slideright/"
X_csvpath = "datasetcsv/landmark.csv"
y_csvpath = "datasetcsv/label.csv"
annotated_path = "tmp/annotated_image2/slideright/"
target = 3

In [184]:
# For static images:
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.2) as hands:

  img_list = os.listdir(data_path)

  for idx, file in enumerate(img_list):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(os.path.join(data_path,file)), 1)
    
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    annotated_image = image.copy()

    if results.multi_hand_landmarks :
      for hand_landmarks in results.multi_hand_landmarks:
        #find out box coordinate
        boundary = calc_bounding_rect(image, hand_landmarks)

        #calculate keypoints coordinate relative 0 keypoints(bottom of palm)
        landmark_list = calc_landmark_list(image,hand_landmarks)
        relative_landmark = pre_process_landmark(landmark_list)

        #append result to csv
        append_to_csv(X_csvpath, relative_landmark)
        append_to_csv(y_csvpath, [target])
            
        #draw box coordinate
        boundary = draw_bounding_rect(True , annotated_image, boundary)

        mp_drawing.draw_landmarks(
              annotated_image,
              hand_landmarks,
              mp_hands.HAND_CONNECTIONS,
              mp_drawing_styles.get_default_hand_landmarks_style(),
              mp_drawing_styles.get_default_hand_connections_style())

    cv2.imwrite(
        annotated_path + str(idx) + '.png', cv2.flip(annotated_image, 1))